In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
% matplotlib inline

In [2]:
import numpy as np
import sys
sys.path.append('..')
from mlscratch.loss_functions import binary_cross_entropy
from mlscratch.activation import *
from mlscratch.nn_layers import fully_connected


class NeuralNetwork(object):

    def __init__(self):
        self.architecture = []
        self.weights = [] # TODO: change to np.array() blank array if possible. # sudo code
        self.local_node_gradients = []
        self.loss_partial_gradients = []
        self.loss_function = None
        self.optimizer = None
        
    def add(self, layer):
        self.architecture.append(layer)

    def compile(self, input_size):
        self.architecture.insert(0, fully_connected(num_nodes=input_size, activation=no_activation))
        for i in range(0, len(self.architecture)-1):
            self.weights.append(self._weight_initialization(self.architecture[i], self.architecture[i+1]))
            
    def train(self, 
              x,
              y, 
              epochs, 
              learning_rate=0.001,
              beta=0.9,
              beta2=0.9,
              velocity=0,
              sqr_velocity=0,
              epsilon=0.000001):
        self.learning_rate = learning_rate
        self.beta = beta
        self.beta2 = beta2
        self.velocity = velocity
        self.epsilon = epsilon
        self.sqr_velocity = sqr_velocity

        for _ in epochs:
            self._forward_pass(input_values=x, y)
            self._backward_pass()


    def validate(self, x, y):
        pass

    def predict(self, x):
        return self._forward_pass(input_values=x)

    def _forward_pass(self, x, y):
        for index, layer_weights in enumerate(self.weights):
            output_values = self.architecture[index+1].activation.evaluate((np.dot(input_values, 
                                                                                   layer_weights.transpose())))
            local_layer_gradients = self.architecture[index+1].activation.derivative(output_values, input_values)  
            self.local_node_gradients.append(local_layer_gradients)
            input_values = output_values
            
        output = input_values        
        return output
    
    def _backward_pass(self):
        for index in range(0, len(self.weights)):
            if index == len(self.weights):
                self.loss_partial_gradients.append(self.weights[index])
            elif index == len(self.weights) - 1:
                node_weights = self.weights[index+1]
                loss_partial_layer_gradients =self.weights[index] * node_weights
                self.loss_partial_gradients.append(loss_partial_layer_gradients)
            else:
                node_weights = self.weights[index+1]
                for index2 in range(index+2, len(self.weights)):
                    node_weights = np.dot(node_weights, self.weights[index2])
                loss_partial_layer_gradients = self.weights[index] * node_weights
                self.loss_partial_gradients.append(loss_partial_layer_gradients)
                
        self.weights, self.velocity, self.sqr_velocity = \
            self.optimizer(theta_array=self.weights,
                           learning_rate=self.learning_rate,
                           beta1=self.beta,
                           beta2=self.beta2,
                           velocity=self.velocity,
                           sqr_velocity=self.sqr_velocity,
                           gradient=self.loss_partial_gradients,
                           epsilon=self.epsilon)
            
    def _weight_initialization(self, layer_l_minus1, layer_l):
        num_nodes_layer_l_minus1 = layer_l_minus1.num_nodes
        num_nodes_layer_l = layer_l.num_nodes
        weight_matrix = np.random.randn(num_nodes_layer_l, num_nodes_layer_l_minus1) * \
                  np.sqrt(2/num_nodes_layer_l_minus1)
        return weight_matrix

In [3]:
# sudo code
from mlscratch.nn_layers import fully_connected
from mlscratch.optimization import adam
from mlscratch.activation import relu, sigmoid
from mlscratch.loss_functions import binary_cross_entropy

neural_network = NeuralNetwork()
neural_network.add(layer=fully_connected(num_nodes=10, activation=relu))
neural_network.add(layer=fully_connected(num_nodes=10, activation=relu))
neural_network.add(layer=fully_connected(num_nodes=1, activation=sigmoid))
neural_network.loss_function = binary_cross_entropy
neural_network.optimizer = adam

In [4]:
x = np.array(np.arange(100))
delta = np.random.uniform(0,10, size=(100,))
y = (.4 * x +1 + delta)

In [5]:
# sudo code
input_size = 100
neural_network.compile(input_size)
neural_network.train(x, y, batch_size=20)
print(x.transpose())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [6]:
neural_network.predict(x)

array([8.22512525e-05])

In [7]:
neural_network.architecture[2].activation

In [13]:
for i in range(0, len(neural_network.local_node_gradients)):
    print(neural_network.local_node_gradients[i+1])

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          2.44518733  0.          0.          2.44518733  2.44518733
   2.44518733  0.          2.44518733  0.        ]
 [ 0.         51.26715052  0.          0.         51.26715052 51.26715052
  51.26715052  0.         51.26715052  0.        ]
 [ 0.         57.35329001  0.          0.         57.35329001 57.35329001
  57.35329001  0.         57.35329001  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         18.91312444  0.          0.         18.91312444 18.91312444
  18.91312444  0.         18.91312444  0.        ]
 [ 0.         19.34789894  0.   

IndexError: list index out of range